In [21]:
from web3 import Web3
from solcx import install_solc, compile_standard
import solcx
import json
import os

install_solc(version='0.8.0')

onchain = Web3(Web3.HTTPProvider('http://localhost:8545')) #on-chain

s1 = Web3(Web3.HTTPProvider('http://localhost:8546')) #shard 1
s2 = Web3(Web3.HTTPProvider('http://localhost:8547')) #shard 2
s3 = Web3(Web3.HTTPProvider('http://localhost:8548')) #shard 3

onchain.isConnected(), s1.isConnected(), s2.isConnected(), s3.isConnected()

(True, True, True, True)

In [22]:
# Finito, si può mettere su python
# DEPLOY DELL'ON CHAIN MANAGER

# SULLA VERSIONE IN PYTHON VERA E PROPRIA DA FARE ALL'AVVIO E CONTROLLARE DA FILE SE ESISTE GIà

path_on_chain = "prova_on_chain.sol"
# Leggo il file contenente lo smart contract
with open(path_on_chain, "r") as file:
    text_file = file.read()

solcx.install_solc('0.8.0')

# Compilo il file contenente lo smart contract
compiled_sol = compile_standard(
        {
            "language": "Solidity",
            "sources": {"prova_on_chain.sol": {"content": text_file}},
            "settings": {
                "outputSelection": {
                    "*": {
                        "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"] # output needed to interact with and deploy contract 
                    }
                }
            },
        },
        solc_version='0.8.0',
    )

# get bytecodes
bytecode = compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["metadata"])["output"]["abi"]

# seleziono l'account di default
# DA MODIFICARE METTENDO GLI ACCOUNT INSERITI DALL'UTENTE
onchain.eth.default_account = onchain.eth.accounts[0]
s1.eth.default_account = s1.eth.accounts[0]
s2.eth.default_account = s2.eth.accounts[0]
s3.eth.default_account = s3.eth.accounts[0]

# deploy dello smart contract dell'onchain manager
On_Chain_Manager = onchain.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = On_Chain_Manager.constructor().transact()
tx_receipt = onchain.eth.wait_for_transaction_receipt(tx_hash)

# creazione dell'oggetto contract per lo smart contract dell'onchain manager in modo da poterci interagire
on_chain_manager = onchain.eth.contract(
     address=tx_receipt.contractAddress,
     abi=abi
)

In [35]:
# codice per far inserire all'utente il path dello smart contract da deployare

print("Inserisci la path del file solidity dello smart contract di cui fare il deploy")
file_path = input()

# controllo che il file esista effettivamente e che sia di tipo .sol
while not os.path.exists(file_path) and os.path.splitext(file_path)[1] != ".sol":
    os.system('clear')
    print("La path deve esistere e il file deve essere di tipo .sol")
    file_path = input()
    
# Leggo il file contenente lo smart contract
with open(file_path, "r") as file:
    text_file = file.read()

Inserisci la path del file solidity dello smart contract di cui fare il deploy


In [36]:
# codice per ottenere solo con il nome del file
import ntpath
file_name = ntpath.basename(file_path)
print(file_name)

SimpleStorage.sol


In [37]:
# codice per leggere la versione di solidity
import re

# espressione regolare utilizzata per trovare dentro allo smart contract la versione di solidity richiesta
regular_expression = "pragma solidity .*;"
# ricerca della stringa pragma dentro lo mart contract tramite espressione regolare
match = re.search(regular_expression, text_file)
# caratteri da rimuovere per rimanere con la stringa composta solo dalla versione di solidity
chars_to_rempove = ['>', '^', '=', ';']
# rimozione dei caratteri
text_version = match.group().split()[2].translate(str.maketrans('','',''.join(chars_to_rempove)))

print(text_version)


0.6.0


In [38]:
# codice per leggere gli import
import re

# espressione regolare utilizzata per trovare dentro allo smart contract la versione di solidity richiesta
regular_expression = "import .*;"
# ricerca della stringa pragma dentro lo mart contract tramite espressione regolare
match = re.findall(regular_expression, text_file)
# caratteri da rimuovere per rimanere con la stringa composta solo dalla versione di solidity
chars_to_rempove = [';', '"']
# rimozione dei caratteri
imports = []
for item in match:
    import_text = item.split()[1]
    import_text = import_text.translate(str.maketrans('','',''.join(chars_to_rempove)))
    imports.append(import_text)

print(imports)

[]


In [39]:
try:
    # Installo la versione del compilatore richiesta dallo smart contract
    solcx.install_solc(text_version)

    #compilo il file
    compiled_solidity = solcx.compile_files([file_path], solc_version=text_version)

    # estraggo i bytecode degli smart contract compilati
    bytecodes = []
    for key in compiled_solidity.keys():
        bytecodes.append(compiled_solidity[key]['bin'])

    # estraggo le abi degli smart contract
    abis = []
    for key in compiled_solidity.keys():
        abis.append(json.loads(json.dumps(compiled_solidity[key]))['abi'])
    print(abis)

except Exception as exception: 
    os.system('clear')
    #viene preso l'output restituito da solc e stampata sulla console solo l'informazione di interesse. 
    jsonError = json.loads(json.dumps(exception.__dict__))
    #items = jsonError["error_dict"][0]
    print("Attenzione! E' stato generato il seguente errore: \n" + jsonError["stderr_data"]
          + "In "+ "\"" +jsonError["command"][1]+ "\""+
          ", con return code: " + str(jsonError["return_code"])+ ".")
    #return


[[{'inputs': [{'internalType': 'string', 'name': '_name', 'type': 'string'}, {'internalType': 'uint256', 'name': '_favoriteNumber', 'type': 'uint256'}], 'name': 'addPerson', 'outputs': [], 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [{'internalType': 'string', 'name': '', 'type': 'string'}], 'name': 'nameToFavoriteNumber', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'stateMutability': 'view', 'type': 'function'}, {'inputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'name': 'people', 'outputs': [{'internalType': 'uint256', 'name': 'favoriteNumber', 'type': 'uint256'}, {'internalType': 'string', 'name': 'name', 'type': 'string'}], 'stateMutability': 'view', 'type': 'function'}, {'inputs': [], 'name': 'retrieve', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'stateMutability': 'view', 'type': 'function'}, {'inputs': [{'internalType': 'uint256', 'name': '_favoriteNumber', 'type': 'uint256'}], '

In [40]:
print(bytecodes)

['608060405234801561001057600080fd5b506105aa806100206000396000f3fe608060405234801561001057600080fd5b50600436106100575760003560e01c80632e64cec11461005c5780636057361d1461007a5780636f760f41146100a85780638bab8dd51461016d5780639e7a13ad1461023c575b600080fd5b6100646102ea565b6040518082815260200191505060405180910390f35b6100a66004803603602081101561009057600080fd5b81019080803590602001909291905050506102f3565b005b61016b600480360360408110156100be57600080fd5b81019080803590602001906401000000008111156100db57600080fd5b8201836020820111156100ed57600080fd5b8035906020019184600183028401116401000000008311171561010f57600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290803590602001909291905050506102fd565b005b6102266004803603602081101561018357600080fd5b81019080803590602001906401000000008111156101a057600080fd5b8201836020820111156101b257600080fd5b803590602001918460018302840111640100000000831117156101d457600080

In [41]:
# Se gli smart contract presenti nel file .sol sono più di uno bisgona chiedere all'utente quali lui voglia deployare

if len(bytecodes) > 1:
    os.system('clear')
    print("Nel file sol appena compilato sono presenti più di uno smart contract. \n"
          + "Gli smart contract compilati sono: "
    )

    # inizializzo il vettore delle possibili selezioni
    possibili_selezioni = []

    # Stampo i nomi degli smart contract compilati con accanto un indice per la selezione
    # Lo stesso indice viene inserito nell'array delle possibili selezioni se si riferisce ad uno smart contract che
    # non sia un abstract contract o un interfaccia
    for index, key in enumerate(compiled_solidity.keys()):
        if bytecodes[index] != '':
            print(str(index) + ') ' + str(key).split(':')[1])
            possibili_selezioni.append(str(index))
        else:
            print(str(index) + ') ' + str(key).split(':')[1] + ' [non è possibile effettuarne il deploy]')
    print("NB: Delle interfacce e gli abstract contract non può essere effettuato il deploy.")
    print("Di quale smart contract vuoi fare il deploy? [inserire il numero di fianco al nome dello smart contract]")

    selezione =input()

    # controllo che l'utente abbia selezionato un indice possibile
    while str(selezione) not in possibili_selezioni:
        print("Inserisci un indice tra quelli degli smart contract di cui è possibile fare il deploy")
        selezione = input()
    
    indice_selezionato = int(selezione)
    abi_to_deploy = abis[indice_selezionato]
    bytecode_to_deploy = bytecodes[indice_selezionato]

else:
    if bytecodes[0] == '':
        os.system('clear')
        print("Lo smart contract presente nel file .sol è una interfeccia o un contratto astratto, \n"
              + "non è quindi possibile effettuarno il deploy"
        )
        # return
    else:
        abi_to_deploy = abis[0]
        bytecode_to_deploy = bytecodes[0]

# controllino in più che lo smart contract selezionato si effettivamente deployabiles
if bytecode_to_deploy == '':
    print('Errore, dello smart contract da te selezionato non può essere fatto il deploy')
    # return
    

In [42]:
try:
     # richiesta all'on-chain-manager su quale shard deployare lo smart contract
     shard_where_deploy = on_chain_manager.functions.get_shard_where_deploy().call()
     print(shard_where_deploy)
except Exception as exception:
     os.system('clear')
     print("Errore: ")
     print(exception)
     print("La richiesta, allo smart contract on-chain manager, riguardo la shard su cui fare il deploy dello smart contract non è andata a buon fine")
     # return


2


In [43]:
# controllo aggiuntivo che non fa mai male
if shard_where_deploy<1 or shard_where_deploy>3:
     print(
          "Errore: \n"
          + "La shard su cui effettuare il deploy dello smart contract deve essere compresa tra 1 e 3"
     )

# switch in base alla shard sulla quale effettuare il deploy
if shard_where_deploy == 1:
     blockchain_where_deploy = s1
elif shard_where_deploy == 2:
     blockchain_where_deploy = s2
elif shard_where_deploy == 3:
     blockchain_where_deploy = s3

In [44]:
# stima del gas necessario per fare il deploy dello smart contract e per aggiornare l'on-chain manager
contratto = blockchain_where_deploy.eth.contract(abi=abi_to_deploy, bytecode=bytecode_to_deploy)

# gas price fisso impostato da noi (Gwei)
gas_price_shard = 20
gas_price_onchain = 20

# stima vera e propria del gas necessario per le due transazioni
stima_gas_deploy = contratto.constructor().estimate_gas()
stima_gas_aggiornamento_on_chain = on_chain_manager.functions.register_contract('0x0000000000000000000000000000000000000000', shard_where_deploy).estimate_gas()

# stima dei costi delle due transazioni, in Gwei, tenendo conto del presso fisso definito sopra
stima_costo_deploy = stima_gas_deploy * gas_price_shard
stima_costo_aggiornamento_onchain = stima_gas_aggiornamento_on_chain * gas_price_onchain

# MODIFICARE METTENDO COME ACCOUNT GLI INDIRIZZI DELL'UTENTE SULLE VARIE BLOCKCHAIN
bilancio_onchain_gwei = onchain.eth.get_balance(onchain.eth.accounts[0])/(10^9)
bilancio_shard_gwei = blockchain_where_deploy.eth.get_balance(blockchain_where_deploy.eth.accounts[0])/(10^9)

print("Bilancio wallet utente sulla shard sulla quale fare il deploy: \n" + str(bilancio_shard_gwei)+' Gwei')
print("Bilancio wallet utente sulla blockchain dell'on-chain manager: \n" +str(bilancio_onchain_gwei)+' Gwei')



if (stima_costo_deploy > bilancio_shard_gwei) or (stima_costo_aggiornamento_onchain > bilancio_onchain_gwei):
    os.system('clear')

    if (stima_costo_deploy > bilancio_shard_gwei):
        print(
            "Attenzione: \n"
            + "Il gas a tua disposizione potrebbe non essere sufficiente a completare il deploy dello smart contract sulla shard interessata!"
        )
    if (stima_costo_aggiornamento_onchain > bilancio_onchain_gwei):
        print(
            "Attenzione: \n"
            + "Il gas a tua disposizione potrebbe non essere sufficiente a completare l'aggiornamento dell'on-chain manager!"
        )
    
    print(
        "Se vuoi, puoi comunque procedere e tentare di effettuare il deploy dello smart contract. \n"
        + "Tuttavia se anche solo una delle due transazioni non dovessero andare a buon fine perderai il gas investito. \n"
        + "Vuoi procedere? [s/n]"
    )
    selezione = ''
    selezione = input()
    while (selezione != 's' or selezione != 'n'):
        print("Inserire solo s oopure n!")
        selezione = input()

    # Decommentare il return e cancellare il pass quando verrà messo in una funzione python vera e propria
    if selezione == 'n':
        pass
        # return

Bilancio wallet utente sulla shard sulla quale fare il deploy: 
3.333333333333333e+20 Gwei
Bilancio wallet utente sulla blockchain dell'on-chain manager: 
3.333328964788532e+20 Gwei


In [45]:
# esecuzione della transazione di deploy dello smart contract
try:
     tx_hash = contratto.constructor().transact()
     tx_receipt = blockchain_where_deploy.eth.wait_for_transaction_receipt(tx_hash)
except Exception as exception:
     os.system('clear')
     print("Errore: \n")
     print(exception)
     print("La transazione di deploy dello smart contract non è andata a buon fine")


# creazione dell'oggetto contratto per interagirci
contratto_deployato = blockchain_where_deploy.eth.contract(
     address=tx_receipt.contractAddress,
     abi=abi
)

# stampa dell'indirizzo del contratto appena deployato
print(contratto_deployato.address)


0x03674b3206ba60b8C157D0397e13d75aAD29f7aA


In [46]:
# Registro che il contratto è stato deployato sull'onchain manager
try:
    on_chain_manager.functions.register_contract(contratto_deployato.address, shard_where_deploy).transact()
except Exception as exception:
     os.system('clear')
     print("Errore: \n")
     print(exception)
     print("La transazione di aggiornamento dell'on-chain manager non è andata a buon fine")

# Test per vedere se il prossimo smart contract viene detto di deployarlo sulla shard dopo
print(on_chain_manager.functions.get_shard_where_deploy().call())

3
